# Pruebas de hipotesis

En este notebook vamos a demostrar pruebas de hipotesis formales usando la data de NHANES.


Es importante notar que la data de NHANES es data de una "encuesta compleja". La data no es independiente ni una muestra representativa de la poblacion target. Analisis apropiado de encuestas complejas debe hacer uso de la informacion adicional sobre como se obtuvo la data. Dado que las encuestas complejas es un tema especializado, vamos a ignorar este aspecto de la data aqui y analizar NHANES como si fuera data independiente y identicamente distribuida (iid) de una muestra de la poblacion.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg') # workaround, there may be a better way
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats.distributions as dist

In [2]:
url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

da["SMQ020x"] = da.SMQ020.replace({1: "Yes", 
                                   2: "No", 
                                   7: np.nan, 
                                   9: np.nan})

In [3]:
da["SMQ020x"].head()

0    Yes
1    Yes
2    Yes
3     No
4     No
Name: SMQ020x, dtype: object

In [4]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

da["RIAGENDRx"].head()

0      Male
1      Male
2      Male
3    Female
4    Female
Name: RIAGENDRx, dtype: object

### Prueba de hipotesis para una proporcion

La prueba de hipotesis mas basica es la de una proporcion para una muestra. Esta prueba se usa si hemos especificado un valor en particular como el valor nulo para la proporcion y deseamos evaluar si la data es compatible con que el parametro real sea igual al valor especificado. Para ilustrarlo, imaginemos que la tasa de _lifetime smoking_ en otro pais es de 40% y queremos evaluar si la tasa de _lifetime smoking_ en los EEUU es diferente a 40%. En las siguientes celdas vamos a realizar una prueba de una muestra (de dos lados) para evaluar si la proporcion poblacional de fumadores es 0.4, y obtener un p-value de 0.43. Esto nos indica que la data de NHANES es compatible con que la proporcion de fumadores en EEUU sea de 40%.

In [5]:
x = da.SMQ020x.dropna() == "Yes"

In [6]:
p = x.mean()

In [7]:
p

0.4050655021834061

In [8]:
se = np.sqrt(.4 * (1 - .4)/ len(x))
se

0.00647467353462031

In [9]:
test_stat = (p - 0.4) / se
test_stat

0.7823563854332805

In [10]:
pvalue = 2 * dist.norm.cdf(-np.abs(test_stat))
print(test_stat, pvalue)

0.7823563854332805 0.4340051581348052


Las siguientes celdas realizan la misma prueba de arriba usando la libreria de `Statsmodels`. Los resultados en el primer caso de abajo son un poco diferentes a los resultados obtenidos arriba porque `Statsmodels` usa por default la proporcion muestral en vez de de la proporcion nula cuando calcula el error estandar. Esta diferencia rara vez tiene consecuencias, pero podemos especificar que se debe usar la proporcion nula para calcular el error estandar, y los resultados son exactamente los mismos a los que calculamos arriba. Las primeras dos lineas realizan la prueba usando la aproximacion normal a la distribucion muestral de la estadistica de prueba, y la tercera linea utiliza la distribucion muestral binomial. Podemos ver que los p-values son casi igual en los 3 casos. Esto es esperado cuando el tamanio de la muestra es grannde y la proporcion no esta cerca de 0 o 1.

In [11]:
sm.stats.proportions_ztest(x.sum(), len(x), 0.4)

(0.7807518954896244, 0.43494843171868214)

In [12]:
sm.stats.binom_test(x.sum(), len(x), 0.4)

0.4340360854459431

### Prueba de hipotesis para dos proporciones

Las pruebas comparativas tienden a ser usadas con mas frecuencia que las pruebas comparando una poblacion con un valor fijo. Una prueba de dos proporciones muestrales se usa para evaluar si la proporcion de individuos con alguna caracteristica difiere entre dos sub-poblaciones. Por ejemplo, podemos comparar las tasas de fumar entre mujeres y hombres. Dado que las tasas de fumar varian fuertemente con la edad, podemos hacer esto en la sub poblacion de personas entre 20 y 25 anios. En las celdas de abajo vamos a realizar esta prueba sin usar la libreria de `Statsmodels`, implementando todos los procedimientos de la prueba que vimos durante el curso. Podemos ver que la tasa de fumar para hombres es ~10 puntos porcentuales mayor a la tasa de fumar para mujeres, y esta diferencia es estadisticamente significativa (p-value ~= 0.01).

In [13]:
dx = da[["SMQ020x", "RIDAGEYR", "RIAGENDRx"]].dropna()

dx.head()

,SMQ020x,RIDAGEYR,RIAGENDRx
0,Yes,62,Male
1,Yes,53,Male
2,Yes,78,Male
3,No,56,Female
4,No,42,Female


In [14]:
p = (dx.groupby("RIAGENDRx")["SMQ020x"]
     .agg([lambda z: np.mean(z == "Yes"), "size"]))

p.columns = ["Smoke", "N"]
print(p)

              Smoke     N
RIAGENDRx                
Female     0.304845  2972
Male       0.513258  2753


Esencialmente las mismas pruebas que arriba se pueden realizar convirtiendo el "yes"/"no" a numeros y realizando un t-test de dos muestras, como abajo:

In [15]:
p_comb = (dx.SMQ020x == "Yes").mean()
va = p_comb * (1 - p_comb)

se = np.sqrt(va * (1 / p.N.Female + 1 / p.N.Male))

In [16]:
(p_comb, va, se)

(0.4050655021834061, 0.2409874411243111, 0.01298546309757376)

In [17]:
test_stat = (p.Smoke.Female - p.Smoke.Male) / se
p_value = 2 * dist.norm.cdf(-np.abs(test_stat))
(test_stat, p_value)

(-16.049719603652488, 5.742288777302776e-58)

In [19]:
dx_females = (dx.loc[dx.RIAGENDRx == "Female", "SMQ020x"]
              .replace({"Yes": 1, "No": 0}))

dx_females.head()

3     0
4     0
5     0
7     0
12    1
Name: SMQ020x, dtype: int64

In [20]:
dx_males = (dx.loc[dx.RIAGENDRx == "Male", "SMQ020x"]
            .replace({"Yes": 1, "No": 0}))

dx_males.head()

0    1
1    1
2    1
6    1
8    0
Name: SMQ020x, dtype: int64

In [21]:
sm.stats.ttest_ind(dx_females, dx_males)

(-16.42058555898443, 3.032088786691117e-59, 5723.0)

### Prueba de hipotesis comparando dos medias

Pruebas de medias son similares a las pruebas de proporciones. Igual que con las proporciones, para comparar medias hay pruebas de una y dos muestras, z-tests y t-tests, de un lado y de dos lados. Igual que con las pruebas de proporciones, las pruebas de una muestra no se usan muy seguido, pero realizamos una abajo. Comparamos systolic blood pressure con el valor fijo de 120 (que es el limite bajo para "pre-hipertension") y encontramos que la media es significativamente diferente de 120 (la estimacion de la media es 126).

In [23]:
dx = da[["BPXSY1", "RIDAGEYR", "RIAGENDRx"]].dropna()
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDRx
0,128.0,62,Male
1,146.0,53,Male
2,138.0,78,Male
3,132.0,56,Female
4,100.0,42,Female


In [24]:
dx = dx.loc[(dx.RIDAGEYR >= 40) & 
            (dx.RIDAGEYR <= 50) & 
            (dx.RIAGENDRx == "Male"), :]
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDRx
10,144.0,46,Male
11,116.0,45,Male
20,110.0,49,Male
42,128.0,42,Male
51,118.0,50,Male


In [25]:
print(dx.BPXSY1.mean())

125.86698337292161


In [26]:
sm.stats.ztest(dx.BPXSY1, value=120)

(7.469764137102597, 8.033869113167905e-14)

En las celdas de abajo, realizamos una prueba formal de la hipotesis nula que la presion arterial media para mujeres entre 50 y 60 anios es igual a la presion arterial media para hombres entre 50 y 60 anios. Los resultados indican que mientras que la presion arterial media de los hombres esta un poco mas arriba que la de las mujeres (129 mm/Hg versus 128 mm/Hg), esta diferencia no es estadisticamente significativa.

Hay diferentes variaciones de pruebas t-test de dos muestras. Dos variaciones que se usan con frecuencia son el t-test realizado con una distribucion t y el t-test realizado usando la aproximacion normal a la distribucion de referencia del estadistico de prueba, a menudo llamado un z-test. Abajo vemos los resultados de estos dos enfoques. Cuando el tamanio de la muestra es grande, la diferencia entre un t-test y un z-test es bastante pequenia.

In [27]:
dx = da[["BPXSY1", "RIDAGEYR", "RIAGENDRx"]].dropna()
dx = dx.loc[(dx.RIDAGEYR >= 50) & (dx.RIDAGEYR <= 60), :]
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDRx
1,146.0,53,Male
3,132.0,56,Female
9,178.0,56,Male
15,134.0,57,Female
19,136.0,54,Female


In [28]:
bpx_female = dx.loc[dx.RIAGENDRx=="Female", "BPXSY1"]
bpx_male = dx.loc[dx.RIAGENDRx=="Male", "BPXSY1"]
print(bpx_female.mean(), bpx_male.mean())

127.92561983471074 129.23829787234044


In [29]:
print(sm.stats.ztest(bpx_female, bpx_male))

(-1.105435895556249, 0.2689707570859362)


In [30]:
print(sm.stats.ttest_ind(bpx_female, bpx_male))

(-1.105435895556249, 0.26925004137768577, 952.0)
